# Stupid Art Project 2017 - Random Holy Text
To fit the stupid art theme "Religion" and my current focus on machine learning, I'm going to generate some random 

The code below was picked up from this site:
http://agiliq.com/blog/2009/06/generating-pseudo-random-text-with-markov-chains-u/
It'll be my first attempt at random holy text generation.

In [1]:
import random


class Markov(object):
    def __init__(self, open_file):
        self.cache = {}
        self.open_file = open_file
        self.words = self.file_to_words()
        self.word_size = len(self.words)
        self.database()

    def file_to_words(self):
        self.open_file.seek(0)
        data = self.open_file.read()
        words = data.split()
        return words

    def triples(self):
        """ Generates triples from the given data string. So if our string were
                "What a lovely day", we'd generate (What, a, lovely) and then
                (a, lovely, day).
        """

        if len(self.words) < 3:
            return

        for i in range(len(self.words) - 2):
            yield (self.words[i], self.words[i + 1], self.words[i + 2])

    def database(self):
        for w1, w2, w3 in self.triples():
            key = (w1, w2)
            if key in self.cache:
                self.cache[key].append(w3)
            else:
                self.cache[key] = [w3]

    def generate_markov_text(self, size=25):
        seed = random.randint(0, self.word_size - 3)
        seed_word, next_word = self.words[seed], self.words[seed + 1]
        w1, w2 = seed_word, next_word
        gen_words = []
        for i in range(size):
            gen_words.append(w1)
            w1, w2 = w2, random.choice(self.cache[(w1, w2)])
        gen_words.append(w2)

        return ' '.join(gen_words)

I'll use the old testament first. I got the text from project Gutenberg, and have removed the Gutenberg pre and postamble. I suspect the text will be too big to run efficiently in the code above, as it is just straight python. We shall see.

In [2]:
with open('sources/old testament.txt') as bible:
    bible_gen = Markov(bible)

In [3]:
bible_gen.generate_markov_text()

'not seeking mine own nation at Jerusalem, Let the floods compassed me about: all ye heathen, and disperse them through the mountains shall they come say'

In [4]:
%%timeit
bible_gen.generate_markov_text()

10000 loops, best of 3: 41.8 µs per loop


It works, and it is fast. But the punctuation and the passage numbering are not the best. I'll try post processing the responses.

Lets first treat the passage numbers appearing mid passage, things like '7:12'. 

In [5]:
import re

'7:12' appearing at the beginning of a passage is desirable actually, so I'd liek to keep those, and possibly force those later. It is the other passage numbers appearing in the middle or end of passages that we want to avoid. Those patterns are guaranteed to have a space in front of them, ie ' 7:12', so I can regex to find and replace using that pattern.

In [6]:
passage_num_pattern = re.compile(r' \d+:\d+')

In [7]:
for _ in range(3):
    passage = bible_gen.generate_markov_text()
    print(passage)
    passage = passage_num_pattern.sub('', passage)
    print(passage)
    

whatsoever is in thine hand with thy servant. 11:12 Notwithstanding in this place. 16:3 For thus saith the LORD, that he cannot pass; 14:6 Turn thy
whatsoever is in thine hand with thy servant. Notwithstanding in this place. For thus saith the LORD, that he cannot pass; Turn thy
and Haran begat Gazez. 2:47 And the sons of Ishmael, the ruler rise up before the vail of the Hamulites. 26:22 These are the sons Heth,
and Haran begat Gazez. And the sons of Ishmael, the ruler rise up before the vail of the Hamulites. These are the sons Heth,
even his enemies from before thy God, hath anointed thee king over Israel. 15:27 And Baasha the son of his distress did he eat thereof. But
even his enemies from before thy God, hath anointed thee king over Israel. And Baasha the son of his distress did he eat thereof. But


But I want to create random passages, so they need to start with a passage number, and contain a few "complete" sentences, ie they end in a period. I think I need to alter the generating method, and I think the alterations will be particular to the old testament. So I'll extend the Markov class with a generating method specialy for this text of the old testament. 

In [8]:
class OldTestaPassagesMarkov(Markov):
    passage_num_pattern = re.compile(r'\d+:\d+')
    
    def generate_markov_text(self, seed_word='', min_words=25):
        
        # Process a user given seed_word
        seed_word_locations = [idx for idx, word in enumerate(self.words) if word.lower() == seed_word.lower()]
            
        if seed_word_locations:
            seed = random.choice(seed_word_locations)
        else:
            print(seed_word + ' is not in Old Testament')
            seed = random.randint(0, self.word_size - 3)
            
        w1, w2 = self.words[seed], self.words[seed + 1]
        gen_words = [w1, w2]
        # go until we have enough words and end in a period
        while w2[-1] != '.' or len(gen_words) < min_words:
            w1, w2 = w2, random.choice(self.cache[(w1, w2)])
            if passage_num_pattern.findall(w2):
                # Avoid adding passage numbers to the middle of the passage.
                # Also end a sentence when a passage number would have gone in.
                new_w1 = w1.replace(':', '.').replace(';', '.')
                gen_words[-1] = new_w1
            else:                
                gen_words.append(w2)

        return ' '.join(gen_words)

In [9]:
with open('sources/old testament.txt') as bible:
    bible_gen = OldTestaPassagesMarkov(bible)

In [10]:
bible_gen.generate_markov_text('12:13')

'12:13 The king of Assyria to the house of the covenant which I have heard the words which were afar off, he sendeth forth his hand against the king.'